In [1]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer

MODEL_NAME = "ai-forever/FRED-T5-1.7B"

/home/khaymonenko/python3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [3]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

/home/khaymonenko/python3.10/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
from peft import get_peft_model, PeftModelForSeq2SeqLM

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 1,743,892,992 || trainable%: 0.2029335524733848


In [5]:
training_args = TrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

NameError: name 'TrainingArguments' is not defined

In [7]:
from aste.train.data_providers.datasets import MVPGenerativeDataset

In [9]:
MVPGenerativeDataset?

Init signature:
MVPGenerativeDataset(
    data: List[aste.data.common.SampleData],
    tokenizer: transformers.tokenization_utils.PreTrainedTokenizer,
    source_max_length: int,
    target_max_length: int,
    order: Optional[List[str]] = None,
    nlp: str = None,
    **kwargs,
)
Docstring:     
An abstract class representing a :class:`Dataset`.

All datasets that represent a map from keys to data samples should subclass
it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
data sample for a given key. Subclasses could also optionally overwrite
:meth:`__len__`, which is expected to return the size of the dataset by many
:class:`~torch.utils.data.Sampler` implementations and the default options
of :class:`~torch.utils.data.DataLoader`.

.. note::
  :class:`~torch.utils.data.DataLoader` by default constructs a index
  sampler that yields integral indices.  To make it work with a map-style
  dataset with non-integral indices/keys, a custom sampler must be provid

In [13]:
PeftModelForSeq2SeqLM.generate??

Signature: PeftModelForSeq2SeqLM.generate(self, **kwargs)
Docstring: <no docstring>
Source:   
    def generate(self, **kwargs):
        peft_config = self.active_peft_config
        self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generation
        self.base_model._prepare_encoder_decoder_kwargs_for_generation = (
            self._prepare_encoder_decoder_kwargs_for_generation
        )
        try:
            if not peft_config.is_prompt_learning:
                with self._enable_peft_forward_hooks(**kwargs):
                    kwargs = {k: v for k, v in kwargs.items() if k not in self.special_peft_forward_args}
                    outputs = self.base_model.generate(**kwargs)
            else:
                if "input_ids" not in kwargs:
                    raise ValueError("input_ids must be provided for Peft model generation")
                if kwargs.get("position_ids", None) is not None:
                    warnings.warn(
                        "Posi